# Motion Profile Velocity
Plot of motion profile velocity, setpoint velocity with distance error P, actual velocity from wheel zero.

In [ ]:
import psycopg2 as pg
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
connection = pg.connect("dbname=jeff user=jeff")

In [ ]:
query = """
SELECT t.millis, u.*
FROM tc_activity a, tc_trace t, unnest(a.trace_measures, t.data) AS u(measure, value)
WHERE a.id = t.activity_id AND a.id = 149;
"""

trace_df = pd.read_sql(query, con=connection)

In [ ]:
vel_df = trace_df.pivot(index='millis', columns='measure', values='value')

vel_df['actual_vel'] = vel_df['actual_vel'].abs()
vel_df['setpoint_vel'] = vel_df['setpoint_vel'] / 10.0
vel_df['profile_vel'] = vel_df['profile_vel'] / 10.0
vel_df['ticks_error'] = vel_df['actual_ticks'] - vel_df['profile_ticks']

In [ ]:
grid=False
fig, (vel_ax, ticks_ax) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(11, 11))
vel_ax.set(title='Velocity', ylabel='ticks/100ms')
ticks_ax.set(title='Distance Error', ylabel='ticks')
vel_df[['profile_vel', 'setpoint_vel', 'actual_vel']].plot(ax=vel_ax, grid=grid)
vel_df[['ticks_error']].plot(ax=ticks_ax, grid=grid, legend=False)